In [ ]:
"""
一维卷积神经网络(1D CNN)用于文本序列分类
================================================

本示例使用一维卷积神经网络对IMDB电影评论进行情感分类。
一维卷积可以有效提取局部特征模式，在文本分类任务中表现出色。

网络架构：
1. Embedding层：将词索引转换为密集向量
2. Conv1D层：提取局部n-gram特征
3. MaxPooling层：降采样，保留重要特征
4. 重复Conv1D和Pooling：提取更高层次特征
5. GlobalMaxPooling：提取全局最显著特征
6. Dense层：二分类输出

数据集：IMDB电影评论数据集（25000条训练样本，25000条测试样本）
"""

import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

# ==================== 数据加载与预处理 ====================

# 参数设置
max_features = 10000  # 词汇表大小，只保留最常见的10000个词
maxlen = 500          # 序列最大长度，超过则截断，不足则填充

# 加载IMDB数据集
print("正在加载IMDB数据集...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 序列填充：将所有序列填充到相同长度
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

print(f"训练样本数量: {len(x_train)}")
print(f"测试样本数量: {len(x_test)}")
print(f"序列形状: {x_train.shape}")

# ==================== 构建一维卷积神经网络 ====================

model = Sequential([
    # 词嵌入层：将词索引映射为128维密集向量
    layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    
    # 第一层卷积：32个7-gram卷积核，提取局部特征
    # 卷积窗口大小为7，可以捕获连续7个词的语义信息
    layers.Conv1D(filters=32, kernel_size=7, activation='relu'),
    
    # 最大池化层：窗口大小为5，降低特征维度
    layers.MaxPooling1D(pool_size=5),
    
    # 第二层卷积：继续提取更高层次的特征
    layers.Conv1D(filters=32, kernel_size=7, activation='relu'),
    
    # 全局最大池化：对每个特征图取最大值，得到固定长度的特征向量
    layers.GlobalMaxPooling1D(),
    
    # 输出层：sigmoid激活函数用于二分类
    layers.Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 显示模型架构
model.summary()

# ==================== 模型训练 ====================

print("\n开始训练模型...")
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.2,  # 使用20%的训练数据作为验证集
    verbose=1
)

# ==================== 可视化训练过程 ====================

# 提取训练历史
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs_range = range(1, len(loss) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制损失曲线
ax1.plot(epochs_range, loss, 'bo-', label='训练损失', linewidth=2)
ax1.plot(epochs_range, val_loss, 'ro-', label='验证损失', linewidth=2)
ax1.set_title('训练和验证损失曲线', fontsize=14, fontweight='bold')
ax1.set_xlabel('轮次', fontsize=12)
ax1.set_ylabel('损失', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制准确率曲线
ax2.plot(epochs_range, acc, 'bo-', label='训练准确率', linewidth=2)
ax2.plot(epochs_range, val_acc, 'ro-', label='验证准确率', linewidth=2)
ax2.set_title('训练和验证准确率曲线', fontsize=14, fontweight='bold')
ax2.set_xlabel('轮次', fontsize=12)
ax2.set_ylabel('准确率', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==================== 模型评估 ====================

print("\n在测试集上评估模型...")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"测试损失: {test_loss:.4f}")
print(f"测试准确率: {test_acc:.4f}")

# 性能分析
print("\n" + "="*50)
print("模型性能分析:")
print("="*50)
print(f"最终训练准确率: {acc[-1]:.4f}")
print(f"最终验证准确率: {val_acc[-1]:.4f}")
print(f"测试准确率: {test_acc:.4f}")
print(f"过拟合程度: {(acc[-1] - val_acc[-1]):.4f}")
print("="*50)